In [ ]:
import signal
from pathlib import Path
import wget

parser_version = '1.6.7'
url = f'https://github.com/nemoware/document-parser/releases/download/{parser_version}/document-parser-{parser_version}.jar'
if not Path(f'document-parser-{parser_version}.jar').is_file():
    wget.download(url)

In [ ]:
import glob
import zipfile

with zipfile.ZipFile(f'./{glob.glob("ДД по практикам*.zip")[0]}',
                     'r') as zip_ref:
    zip_ref.extractall('./')

In [ ]:
key_value = ['о нижеследующем:', 'нижеследующем:', 'о нижеследующем', 'нижеследующем']
for x in key_value[5:]:
    print(123)


In [1]:
parser_version = '1.6.7'
!java -cp "document-parser-$parser_version/classes;document-parser-$parser_version/lib/*" com.nemo.document.parser.App -i "Документы\ДД по практикам\Практика недропользования и экологии\ДС.docx"

{"documents":[{"documentType":"SUPPLEMENTARY_AGREEMENT","paragraphs":[{"paragraphHeader":{"offset":0,"text":"ДОПОЛНИТЕЛЬНОЕ СОГЛАШЕНИЕ №1","length":28},"paragraphBody":{"offset":28,"text":"к Договору № 12/19-П от 12.12.2020 г.\r\nг. Н.Новгород                                                                                                               «01» июня 2021 г.\r\nОбщество с ограниченной ответственностью «Петухов», именуемое в дальнейшем «Поставщик», в лице директора Крылова Александра Сергеевича, действующего на основании Устава, с одной стороны и \r\nОбщество с ограниченной ответственностью «План-НН», именуемое в дальнейшем «Покупатель», в лице Сальникова Евгения Константиновича, действующего на основании Доверенности № 01/11 от 21.11.2020 г., с другой стороны, вместе именуемые Стороны, заключили настоящее дополнительное соглашение к Договору № 12/19-П от 12.12.2020 г. о нижеследующем:\r\n1. п.1.1. Договора №12/19-П от 12.12.2020 г. изложить в следующей редакции:\r\n«1.1. Пос

In [ ]:
import subprocess

s = [
    "gshell",
    "init"
]

subprocess.run(s, stdin=subprocess.PIPE, stdout=subprocess.PIPE)

# Start

## Import

In [2]:
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import platform
import sys
import os
import json
import subprocess
import pandas as pd
import fnmatch
import base64
import requests
import time
import numpy as np
import importlib
import search_text

## Create list of all docs path

In [3]:
arrOfAllDocs = []

for root, dir_names, filenames in os.walk('Документы'):
    if len(root.split('\\')) == 2: continue
    flag = False
    for i in root.split('\\'):
        if str(i).startswith('Исключена'):
            flag = True
            break
    if flag: continue
    for filename in fnmatch.filter(filenames, '*.docx'):
        arrOfAllDocs.append(os.path.join(root, filename))
    for filename in fnmatch.filter(filenames, '*.doc'):
        arrOfAllDocs.append(os.path.join(root, filename))


In [4]:
arrOfAllDocs[:]

['Документы\\Входящие по практикам\\Практика коммерческой логистики\\01-Шаблон Исковое заявление Типовой договор поставки ГПН (штраф за простой вагонов).docx',
 'Документы\\Входящие по практикам\\Практика коммерческой логистики\\Акт общей формы_ШАБЛОН_ЛЕД_дополнен.docx',
 'Документы\\Входящие по практикам\\Практика коммерческой логистики\\АКТ ОТБОРА ПРОБ_ШАБЛОН.docx',
 'Документы\\Входящие по практикам\\Практика коммерческой логистики\\АКТ ПРИЕМКИ по КАЧЕСТВУ_ШАБЛОН.docx',
 'Документы\\Входящие по практикам\\Практика коммерческой логистики\\Заявление ССП_шаблон на бланке ПАО.docx',
 'Документы\\Входящие по практикам\\Практика коммерческой логистики\\ПРЕТЕНЗИЯ_излишки ГПН - ГПН-РП_ШАБЛОН для Регламента.docx',
 'Документы\\Входящие по практикам\\Практика коммерческой логистики\\ПРЕТЕНЗИЯ_недостача ГПН - ГПН-РП_ШАБЛОН для Регламента.docx',
 'Документы\\Входящие по практикам\\Практика коммерческой логистики\\расчет претензии ГПН-Л к РЖД_ШАБЛОН.docx',
 'Документы\\Входящие по практикам\\Пра

In [12]:
importlib.reload(search_text)

index = 1
number_of_docs = 0
parser_version = '1.6.7'

tokenizer_path = "sberbank-ai/ruRoberta-large"

labels = ['Практика коммерческой логистики',
          'Практика недропользования и экологии',
          'Практика поддержки региональных, розничных продаж и клиентского сервиса',
          'Практика правового сопровождения закупок МТР и услуг общего профиля',
          'Практика правового сопровождения земельных отношений и сделок с недвижимым имуществом',
          'Практика правового сопровождения операционной деятельности БРД',
          'Практика правового сопровождения переработки и инфраструктуры',
          'Практика правовой поддержки брендов',
          'Практика правовой поддержки использования и коммерциализации ИС',
          'Практика правовой поддержки создания и приобретения ИС',
          'Практика промышленной безопасности и охраны труда',
          'Практика финансового и конкурентного права',
          'Практика экспорта, оптовых продаж и сбыта бизнес-единиц (БЕ)']

result = []
result_of_fail = []
result_of_possible = []
result_of_possible2 = []

s = [
    "java",
    "-jar",
    f"document-parser-{parser_version}.jar",
    "--server.port=8083"
]
headers = {
    'Content-type': 'application/json',
    'Accept': 'application/json; text/plain'
}

print(
    "Запуск document-parser на 8083 порту, если что-то пойдет не так, то руками УБЕЙТЕ java процесс"
)
java_subprocess = subprocess.Popen(s, creationflags=subprocess.CREATE_NEW_PROCESS_GROUP,
                                   stdout=subprocess.PIPE, encoding="utf-8")

time.sleep(2)
i = 1
while True:
    time.sleep(0.1)
    output_log_spring = java_subprocess.stdout.readline()
    sys.stdout.write("\rПроверка соединения #%i" % i)
    sys.stdout.flush()
    i += 1
    if output_log_spring.find("Started DocumentParserService") != -1:
        print("\nГотово")
        java_subprocess.stdout.close()
        break

print("Запустился успешно")
print("Общее количество документов =", len(arrOfAllDocs))

for docs in arrOfAllDocs:
    try:
        file = open(docs, 'rb')
        encoded_string = base64.b64encode(file.read())
        encoded_string = str(encoded_string)[2:-1]
    except Exception as e:
        # print(f"\nОшибка в файле {docs}")
        # print(f"при конвертации в base64, исключение = {e.msg}")
        # print("=" * 200)
        continue
    is_doc = True
    is_docx = True
    is_not_bad_doc = True
    doc_type = docs.split(".")[-1].upper()
    resArr = []
    document = []
    while is_doc or is_docx or is_not_bad_doc:
        response = requests.post(
            "http://localhost:8083/document-parser",
            data=json.dumps({
                "base64Content": encoded_string,
                "documentFileType": doc_type
            }),
            headers=headers
        )
        if 'message' in response.json() and not (is_doc == False and is_docx == False):
            if doc_type == 'DOC':
                is_doc = False
                doc_type = 'DOCX'
                continue
            if doc_type == 'DOCX':
                is_docx = False
                doc_type = 'DOC'
                continue

        try:
            resArr = response.json()['documents']
        except Exception as e:
            # print(f"\nОшибка в файле {docs}")
            # print(f"Ответ от парсера {response.json()}")
            # print("=" * 200)
            continue
        finally:
            is_doc = False
            is_docx = False
            is_not_bad_doc = False

    sys.stdout.write(
        f"\rNow {index} from {len(arrOfAllDocs)} и Good = {len(result)}, Bad = {len(result_of_fail)}, TEST = {len(result_of_possible)}, TEST2 = {len(result_of_possible2)}")
    sys.stdout.flush()
    index += 1

    if resArr:
        document = resArr[0]
    else:
        continue

    result_of_single_doc, enum = search_text.find_text(document, path=docs)
    if enum == search_text.list_of_sheets.GOOD:
        result.append(result_of_single_doc)
    if enum == search_text.list_of_sheets.BAD:
        result_of_fail.append(result_of_single_doc)
    if enum == search_text.list_of_sheets.TEST:
        result_of_possible.append(result_of_single_doc)
    if enum == search_text.list_of_sheets.TEST2:
        result.append(result_of_single_doc)
        result_of_possible2.append(result_of_single_doc)

#Смерть java процессу!
if platform.system() == 'Windows':
    subprocess.run("TASKKILL /F /PID {pid} /T".format(pid=java_subprocess.pid))
elif platform.system() == 'Linux':
    os.kill(java_subprocess.pid, signal.SIGTERM)
else:
    print('Не известная платформа, убейте в ручную процесс java')
# os.killpg(os.getpgid(java_subprocess.pid), signal.SIGTERM)

writer = pd.ExcelWriter("classifier.xlsx", engine='xlsxwriter')

df = pd.DataFrame(result)
df.to_excel(writer, 'good', engine='xlsxwriter')
sheets_good = writer.sheets['good']
sheets_good.autofilter(0, 0, df.shape[0], df.shape[1])

df = pd.DataFrame(result_of_fail)
df.to_excel(writer, 'SO SO', engine='xlsxwriter')
sheets_bad = writer.sheets['SO SO']
sheets_bad.autofilter(0, 0, df.shape[0], df.shape[1])

df = pd.DataFrame(result_of_possible)
df.to_excel(writer, 'Новые практики', engine='xlsxwriter')
sheets_bad = writer.sheets['Новые практики']
sheets_bad.autofilter(0, 0, df.shape[0], df.shape[1])

df = pd.DataFrame(result_of_possible2)
df.to_excel(writer, 'Результат по новым практикам', engine='xlsxwriter')
sheets_bad = writer.sheets['Результат по новым практикам']
sheets_bad.autofilter(0, 0, df.shape[0], df.shape[1])

writer.save()
print("\nФайл создан")

Запуск document-parser на 8083 порту, если что-то пойдет не так, то руками УБЕЙТЕ java процесс
Проверка соединения #31
Готово
Запустился успешно
Общее количество документов = 2250
Now 2250 from 2250 и Good = 1717, Bad = 60, TEST = 463, TEST2 = 818
Файл создан
